# Team Rating Model 

### Data Cleaning/Preprocessing

In [1]:
import pandas as pd
import numpy as np
import sklearn
import math

/Users/jcs/opt/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#prep the dataset with only the features we want
df=pd.read_csv('Data/AllPlayerYesSalaryEncoded.csv')
df_rating=pd.read_csv('Data/AllPlayerNoSalaryCleaned.csv')
wanted_columns=['Player','Season','ORtg','DRtg',]
df_rating_columns=df_rating.columns
for column in wanted_columns:
    df_rating_columns=df_rating_columns.drop(column)
df_rating=df_rating.drop(columns=df_rating_columns)
#print(df)
df_rating=df_rating.astype({'Season':str,'Player':str,'ORtg':int,'DRtg':int})
df=df.astype({'Season':str,'Tm':str,'Player':str,'feature1':float,'feature2':float})

df_rating['key']=df_rating['Player']+df_rating['Season']
df['key']=df['Player']+df['Season']

print(df_rating['key'])
df_rating=df_rating.drop(columns=['Player','Season'])
new_df=pd.merge(df,df_rating,on='key')
df=new_df
columns=df.columns
#print(columns)
wanted_columns=['Player','Tm','ORtg','DRtg','Season','feature1','feature2']
for column in wanted_columns:
    columns=columns.drop(column)
#print(columns)
df=df.drop(columns=columns)
df


0         Precious Achiuwa2023.0
1             Steven Adams2023.0
2              Bam Adebayo2023.0
3             Ochai Agbaji2023.0
4             Santi Aldama2023.0
                  ...           
10381    Dedric Willoughby2000.0
10382     Haywoode Workman2000.0
10383    Metta World Peace2000.0
10384      Lorenzen Wright2000.0
10385               Tim Youngnan
Name: key, Length: 10386, dtype: object


,Player,Tm,Season,feature1,feature2,ORtg,DRtg
0,Tariq Abdul-Wahad,ORL,2000.0,1.615574,-0.661990,98,103
1,Tariq Abdul-Wahad,DEN,2000.0,1.329859,-0.831828,98,103
2,Shareef Abdur-Rahim,VAN,2000.0,2.207691,-0.742464,108,106
3,Cory Alexander,DEN,2000.0,1.232419,-0.859595,84,103
4,Ray Allen*,MIL,2000.0,1.771979,-1.120610,115,110
...,...,...,...,...,...,...,...
11201,McKinley Wright IV,DAL,2023.0,1.303945,-0.863506,113,118
11202,Thaddeus Young,TOR,2023.0,1.419165,-0.411144,117,112
11203,Trae Young,ATL,2023.0,2.340780,-1.171261,116,119
11204,Cody Zeller,MIA,2023.0,1.622240,0.543657,124,114


In [3]:
df['NetRtg']=df['ORtg']-df['DRtg']
df=df.drop(columns=['ORtg','DRtg'])
df

,Player,Tm,Season,feature1,feature2,NetRtg
0,Tariq Abdul-Wahad,ORL,2000.0,1.615574,-0.661990,-5
1,Tariq Abdul-Wahad,DEN,2000.0,1.329859,-0.831828,-5
2,Shareef Abdur-Rahim,VAN,2000.0,2.207691,-0.742464,2
3,Cory Alexander,DEN,2000.0,1.232419,-0.859595,-19
4,Ray Allen*,MIL,2000.0,1.771979,-1.120610,5
...,...,...,...,...,...,...
11201,McKinley Wright IV,DAL,2023.0,1.303945,-0.863506,-5
11202,Thaddeus Young,TOR,2023.0,1.419165,-0.411144,5
11203,Trae Young,ATL,2023.0,2.340780,-1.171261,-3
11204,Cody Zeller,MIA,2023.0,1.622240,0.543657,10


In [4]:
#list_of_teams=df['Tm'].tolist()
#list_of_teams=set(list_of_teams)
#list_of_teams=list(list_of_teams)
#team_dict={}
#for team in list_of_teams:
#    df_temp=df[df['Tm']==team]
#    df_temp=df_temp.drop(columns=['Tm'])
#    team_dict[team]=df_temp.values.tolist()
def getTopN(team_roster,n):
    top_players=[]
    #print(len(team_roster))
    for i in range(n):
        max_rating=['',float('-inf'),float('-inf'),float('-inf')]
        max_index=0
        for i in range(len(team_roster)):
            player=team_roster[i]
            print(player)
            if player[3]>max_rating[3]:
                max_rating=player
                max_index=i
        team_roster.remove(max_rating)
        top_players.append(max_rating)
    return top_players
#for key,value in team_dict.items():
    #team_dict[key]=getTopN(value,5)
#for key,value in team_dict.items():
    #print(key,value)

In [40]:
df_team=pd.read_csv('Data/TeamData.csv')
#df_team
#converting team name so it lines up
#when player data is in organize it into a dataframe for the 
df_team


,Rk,Year,Team,NRtg
0,1,2023,Boston Celtics,6.70
1,2,2023,Cleveland Cavaliers,5.62
2,3,2023,Philadelphia 76ers,4.35
3,4,2023,Memphis Grizzlies,3.93
4,5,2023,Milwaukee Bucks,3.49
...,...,...,...,...
710,25,2000,Vancouver Grizzlies,-6.17
711,26,2000,Atlanta Hawks,-5.78
712,27,2000,Golden State Warriors,-8.64
713,28,2000,Chicago Bulls,-10.43


In [41]:
df_team
dict_team_names={'Boston Celtics':'BOS',
                 'Cleveland Cavaliers':'CLE',
                 'Philadelphia 76ers':'PHI',
                 'Memphis Grizzlies':'MEM',
                 'Milwaukee Bucks':'MIL',
                 'Vancouver Grizzlies':'VAN',
                 'Atlanta Hawks':'ATL',
                 'Golden State Warriors':'GSW',
                 'Chicago Bulls':'CHI',
                 'Los Angeles Clippers':'LAC',
                 'New York Knicks':'NYK',
                 'Denver Nuggets':'DEN',
                 'Sacramento Kings':'SAC',
                 'Phoenix Suns':'PHO',
                 'Toronto Raptors':'TOR',
                 'New Orleans Pelicans':'NOP',
                 'Brooklyn Nets':'BRK',
                 'Oklahoma City Thunder':'OKC',
                 'Dallas Mavericks':'DAL',
                 'Minnesota Timberwolves':'MIN',
                 'Miami Heat':'MIA',
                 'Utah Jazz':'UTA',
                 'Washington Wizards':'WAS',
                 'Orlando Magic':'ORL',
                 'Indiana Pacers':'IND',
                 'Portland Trail Blazers':'POR',
                 'Charlotte Hornets':'CHO',
                 'Houston Rockets':'HOU',
                 'Detroit Pistons':'DET',
                 'San Antonio Spurs':'SAS',
                 'Los Angeles Lakers':'LAL',
                 }
print(len(dict_team_names))
df_team=df_team.replace({"Team":dict_team_names})
df_team

31


,Rk,Year,Team,NRtg
0,1,2023,BOS,6.70
1,2,2023,CLE,5.62
2,3,2023,PHI,4.35
3,4,2023,MEM,3.93
4,5,2023,MIL,3.49
...,...,...,...,...
710,25,2000,VAN,-6.17
711,26,2000,ATL,-5.78
712,27,2000,GSW,-8.64
713,28,2000,CHI,-10.43


In [26]:
very_temp_df=df[df['Season']==2004.0]
very_temp_df=very_temp_df[very_temp_df['Tm']=='ATL']
df

,Player,Tm,Season,feature1,feature2,NetRtg
0,Tariq Abdul-Wahad,ORL,2000.0,1.615574,-0.661990,-5
1,Tariq Abdul-Wahad,DEN,2000.0,1.329859,-0.831828,-5
2,Shareef Abdur-Rahim,VAN,2000.0,2.207691,-0.742464,2
3,Cory Alexander,DEN,2000.0,1.232419,-0.859595,-19
4,Ray Allen*,MIL,2000.0,1.771979,-1.120610,5
...,...,...,...,...,...,...
11201,McKinley Wright IV,DAL,2023.0,1.303945,-0.863506,-5
11202,Thaddeus Young,TOR,2023.0,1.419165,-0.411144,5
11203,Trae Young,ATL,2023.0,2.340780,-1.171261,-3
11204,Cody Zeller,MIA,2023.0,1.622240,0.543657,10


In [ ]:
years=range(2000,2024)
teams_for_years={}
for year in years:
    year=str(float(year))
    temp_df=df[df['Season']==year]
    #print(temp_df)
    list_of_teams=temp_df['Tm'].tolist()
    list_of_teams=set(list_of_teams)
    list_of_teams=list(list_of_teams)
    team_dict={}
    for team in list_of_teams:
        df_temp=temp_df[temp_df['Tm']==team]
        #print(df_temp)
        df_temp=df_temp.drop(columns=['Season'])
        df_temp=df_temp.drop(columns=['Tm'])
        if len(df_temp.values.tolist())>=8:
            team_dict[team]=df_temp.values.tolist()
            #print(team_dict[team])
    for key,value in team_dict.items():
        #print(value)
        #print(key,year)
        team_dict[key]=getTopN(value,8)
    if team_dict=={}:
        pass
    else:
        teams_for_years[year]=team_dict
print(teams_for_years)

In [28]:
#print(teams_for_years[2000]['MIL'])
#print(teams_for_years[2001]['MIL'])
# look up how to add rows for the thingie and just do that for year for team and 
# that should be easy then concat based on year and team then drop year and team
features=['player1Name','player1Feature1','player1Feature2','player1Rtg',
          'player2Name','player2Feature1','player2Feature2','player2Rtg',
          'player3Name','player3Feature1','player3Feature2','player3Rtg',
          'player4Name','player4Feature1','player4Feature2','player4Rtg',
          'player5Name','player5Feature1','player5Feature2','player5Rtg',
          'player6Name','player6Feature1','player6Feature2','player6Rtg',
          'player7Name','player7Feature1','player7Feature2','player7Rtg',
          'player8Name','player8Feature1','player8Feature2','player8Rtg']
df_dict={'Team':[],'Year':[],
          'player1Name':[],'player1Feature1':[],'player1Feature2':[],'player1Rtg':[],
          'player2Name':[],'player2Feature1':[],'player2Feature2':[],'player2Rtg':[],
          'player3Name':[],'player3Feature1':[],'player3Feature2':[],'player3Rtg':[],
          'player4Name':[],'player4Feature1':[],'player4Feature2':[],'player4Rtg':[],
          'player5Name':[],'player5Feature1':[],'player5Feature2':[],'player5Rtg':[],
          'player6Name':[],'player6Feature1':[],'player6Feature2':[],'player6Rtg':[],
          'player7Name':[],'player7Feature1':[],'player7Feature2':[],'player7Rtg':[],
          'player8Name':[],'player8Feature1':[],'player8Feature2':[],'player8Rtg':[],}
for year in years:
    year=str(float(year))
    for team in list_of_teams:
        try:
            entries=teams_for_years[year][team]
            df_dict['Team'].append(team)
            df_dict['Year'].append(year)
            counter=0
            for j in range(len(entries)):
                for i in range(len(entries[j])):
                    player_number=features[counter]
                    #print(entries[j][i])
                    df_dict[player_number].append(entries[j][i])
                    counter+=1
        except:
            pass
#print(df_dict)
#print(len(df_dict['Team']))
#print(len(df_dict['Year']))
#print(len(df_dict['player1Feature1']))
#print(len(df_dict['player1Feature2']))
#print(len(df_dict['player2Feature1']))
#print(len(df_dict['player2Feature2']))
#print(len(df_dict['player3Feature1']))
#print(len(df_dict['player3Feature2']))
#print(len(df_dict['player4Feature1']))
#print(len(df_dict['player4Feature2']))
#print(len(df_dict['player5Feature1']))
#print(len(df_dict['player5Feature2']))
#print(len(df_dict['player6Feature1']))
#print(len(df_dict['player6Feature2']))
#print(len(df_dict['player7Feature1']))
#print(len(df_dict['player7Feature2']))
#print(len(df_dict['player8Feature1']))
##print(len(df_dict['player8Feature2']))
team_year_player_df=pd.DataFrame(df_dict)
team_year_player_df
        

,Team,Year,player1Name,player1Feature1,player1Feature2,player1Rtg,player2Name,player2Feature1,player2Feature2,player2Rtg,...,player6Feature2,player6Rtg,player7Name,player7Feature1,player7Feature2,player7Rtg,player8Name,player8Feature1,player8Feature2,player8Rtg
0,MIA,2000.0,Duane Causwell,1.477994,0.537829,16,Alonzo Mourning*,2.426133,0.388596,16,...,0.141047,9,Tim Hardaway*,1.686293,-1.129065,1,Jamal Mashburn,1.722943,-1.061987,1
1,SAS,2000.0,David Robinson*,2.278321,0.318209,20,Tim Duncan*,2.523733,-0.399965,14,...,-0.960060,7,Mario Elie,1.203198,-0.980064,3,Avery Johnson,1.816274,-0.914619,3
2,DEN,2000.0,Ryan Bowen,1.095921,-0.477948,9,Bryant Stith,1.098415,-0.872210,5,...,-1.039629,0,Chris Gatling,1.615754,-0.456281,-1,Popeye Jones,1.343768,-0.309056,-1
3,SAC,2000.0,Scot Pollard,1.716204,0.503324,23,Jon Barry,1.226496,-0.996845,14,...,-0.906657,6,Corliss Williamson,1.593672,-0.086515,0,Tony Delk,1.390068,-0.499176,-2
4,POR,2000.0,Arvydas Sabonis*,1.920113,-0.044496,16,Steve Smith,1.501189,-0.957072,16,...,0.281843,6,Scottie Pippen*,1.689561,-0.943810,6,Jermaine O'Neal,1.590377,0.524440,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,UTA,2023.0,Walker Kessler,1.932538,0.247177,29,Udoka Azubuike,1.523804,0.782774,17,...,-0.784937,9,Mike Conley,1.506012,-1.207218,7,Kelly Olynyk,1.723821,-0.638189,3
663,PHI,2023.0,Paul Reed,1.624038,0.198613,17,Joel Embiid,2.662494,-0.389367,15,...,-0.902510,4,Tyrese Maxey,1.586397,-1.099570,3,Matisse Thybulle,0.801173,-0.807810,1
664,MEM,2023.0,Brandon Clarke,1.665634,-0.307017,22,Xavier Tillman Sr.,1.661685,-0.130368,20,...,-0.576006,9,Tyus Jones,1.531711,-1.188774,9,John Konchar,1.014680,-0.877785,8
665,CHI,2023.0,Tony Bradley,1.048575,-0.319352,25,Javonte Green,1.125514,-0.562623,13,...,-1.048127,6,DeMar DeRozan,2.072370,-0.951415,6,Dalen Terry,1.020823,-0.613438,2


In [42]:

df_team['Year']=df_team['Year']+'.0'

df_team['TeamYear']=df_team['Team']+df_team['Year']
df_team=df_team.drop(columns=['Team','Year','Rk'])
df_team

,NRtg,TeamYear
0,6.70,BOS2023.0
1,5.62,CLE2023.0
2,4.35,PHI2023.0
3,3.93,MEM2023.0
4,3.49,MIL2023.0
...,...,...
710,-6.17,VAN2000.0
711,-5.78,ATL2000.0
712,-8.64,GSW2000.0
713,-10.43,CHI2000.0


In [43]:
#dict_type={'Team':str,'Year':str,'player1':int,'player2':int,'player3':int,'player4':int,'player5':int,'player6':int,'player7':int,'player8':int}
#team_year_player_df=team_year_player_df.apply(dict_type)
team_year_player_df['TeamYear']=team_year_player_df['Team']+team_year_player_df['Year']
team_year_player_df=team_year_player_df.drop(columns=['Team','Year'])
team_year_player_df

KeyError: 'Team'

In [44]:
final_df=pd.merge(df_team,team_year_player_df,on='TeamYear')

final_df

,NRtg,TeamYear,player1Name,player1Feature1,player1Feature2,player1Rtg,player2Name,player2Feature1,player2Feature2,player2Rtg,...,player6Feature2,player6Rtg,player7Name,player7Feature1,player7Feature2,player7Rtg,player8Name,player8Feature1,player8Feature2,player8Rtg
0,6.70,BOS2023.0,Robert Williams,1.793262,0.462374,36,Luke Kornet,1.425073,0.117529,32,...,-1.029202,11,Mike Muscala,1.125109,-0.612078,11,Malcolm Brogdon,1.619986,-1.035222,9
1,5.62,CLE2023.0,Jarrett Allen,2.035402,0.116011,24,Raul Neto,1.217779,-0.978066,13,...,-1.128473,7,Darius Garland,2.012217,-1.197642,5,Dean Wade,0.929818,-0.790122,3
2,4.35,PHI2023.0,Paul Reed,1.624038,0.198613,17,Joel Embiid,2.662494,-0.389367,15,...,-0.902510,4,Tyrese Maxey,1.586397,-1.099570,3,Matisse Thybulle,0.801173,-0.807810,1
3,3.93,MEM2023.0,Brandon Clarke,1.665634,-0.307017,22,Xavier Tillman Sr.,1.661685,-0.130368,20,...,-0.576006,9,Tyus Jones,1.531711,-1.188774,9,John Konchar,1.014680,-0.877785,8
4,3.49,MIL2023.0,Jae Crowder,1.144830,-0.852213,22,Brook Lopez,1.749598,-0.315063,13,...,-0.587251,7,A.J. Green,0.738680,-1.146615,6,Jrue Holiday,2.003336,-1.100547,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,-3.81,CLE2000.0,Earl Boykins,1.416882,-0.952905,6,Danny Ferry,1.364945,-0.655821,4,...,0.376578,-2,Mark Bryant,1.613921,0.149142,-3,Shawn Kemp,2.361587,0.065049,-4
663,-5.78,ATL2000.0,Dikembe Mutombo*,2.245130,0.542822,15,Lorenzen Wright,1.568489,-0.255307,0,...,-1.057754,-8,Chris Crawford,1.250908,-0.464399,-9,Jim Jackson,1.614732,-1.005799,-9
664,-8.64,GSW2000.0,Tony Farmer,1.556841,-0.187960,3,Donyell Marshall,1.887353,-0.593936,-3,...,0.570811,-6,Terry Cummings,1.729570,0.108624,-8,Damon Jones,1.218256,-1.101450,-8
665,-10.43,CHI2000.0,Fred Hoiberg,1.225271,-1.000219,2,Elton Brand,2.397093,0.197015,1,...,-0.895599,-8,Dedric Willoughby,1.191200,-1.046972,-10,Kornél Dávid,1.343862,-0.163687,-11


In [45]:
features=['player1Name','player1Rtg',
          'player2Name','player2Rtg',
          'player3Name','player3Rtg',
          'player4Name','player4Rtg',
          'player5Name','player5Rtg',
          'player6Name','player6Rtg',
          'player7Name','player7Rtg',
          'player8Name','player8Rtg','TeamYear']
final_df=final_df.drop(columns=features)

In [47]:
final_df.drop(columns='NRtg')

,player1Feature1,player1Feature2,player2Feature1,player2Feature2,player3Feature1,player3Feature2,player4Feature1,player4Feature2,player5Feature1,player5Feature2,player6Feature1,player6Feature2,player7Feature1,player7Feature2,player8Feature1,player8Feature2
0,1.793262,0.462374,1.425073,0.117529,1.340585,-0.595223,1.193328,-0.323736,1.399907,-1.082706,0.872047,-1.029202,1.125109,-0.612078,1.619986,-1.035222
1,2.035402,0.116011,1.217779,-0.978066,1.029012,-0.845330,2.075747,-0.263675,1.230536,-0.611300,1.962617,-1.128473,2.012217,-1.197642,0.929818,-0.790122
2,1.624038,0.198613,2.662494,-0.389367,1.582267,0.557196,2.159223,-1.233361,1.267492,-0.486682,1.565536,-0.902510,1.586397,-1.099570,0.801173,-0.807810
3,1.665634,-0.307017,1.661685,-0.130368,1.930034,-0.337966,0.982349,-1.290543,2.003255,0.585468,1.361234,-0.576006,1.531711,-1.188774,1.014680,-0.877785
4,1.144830,-0.852213,1.749598,-0.315063,1.096461,-1.082687,2.729928,-0.400123,1.126550,-1.063455,1.753406,-0.587251,0.738680,-1.146615,2.003336,-1.100547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,1.416882,-0.952905,1.364945,-0.655821,1.982237,-0.747579,1.157179,-0.982961,1.502874,-0.983225,1.770005,0.376578,1.613921,0.149142,2.361587,0.065049
663,2.245130,0.542822,1.568489,-0.255307,1.926919,-0.309053,1.491518,-0.611187,1.624929,-0.783768,1.594455,-1.057754,1.250908,-0.464399,1.614732,-1.005799
664,1.556841,-0.187960,1.887353,-0.593936,1.587634,-0.863641,1.138122,-0.846780,1.721529,-1.129734,1.716349,0.570811,1.729570,0.108624,1.218256,-1.101450
665,1.225271,-1.000219,2.397093,0.197015,1.646628,-0.034823,1.255791,-1.019783,1.874744,-0.768303,1.582896,-0.895599,1.191200,-1.046972,1.343862,-0.163687


In [48]:
#save to a csv
final_df.to_csv('Data/TeamNotEncodedData.csv')

In [ ]:
final_df['sum']=final_df['player1']+final_df['player2']+final_df['player3']+final_df['player4']+final_df['player5']+final_df['player6']+final_df['player7']+final_df['player8']
final_df=final_df.drop(columns=features)
final_df
import matplotlib.pyplot as plt
plt.scatter(final_df)
plt.show

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(x=final_df['NRtg'],y=final_df['sum'])
plt.show